In [1]:
import numpy as np
from quantum import quantum

# The Deutsch-Joza Algorithm

We are given a classical black box that takes n input qubits, and outputs to one output qubit.

To make the circuit reversible we need to adapt our black box so it has the same number of input qubits as output qubits. In our case we give the circuit n input qubits plus one extra qubit for output, the circuit returns the original n input qubits in their initial state, plus the output qubit in its new state.

We are guaranteed that the output of this black box is either constant or balanced:

- constant: The state of the output bit is the same for all input states;
- balanced: The output bit is ON for half of the input states and OFF for the other half.

The problem is to find out whether the black box is constant or balanced.

The Deutsch-Joza algorithm is simple:

1. Set the output qubit to |-⟩
2. Apply H-gates to the n input qubits
3. Run the black box
4. Apply H-gates to the n input qubits
5. Measure the n input qubits: if all n qubits are in the state |0⟩, the box is constant; otherwise, the box is balanced.

In [2]:
# Set the output qubit to |-⟩ = HX|0⟩
output = np.dot(quantum.H, np.dot(quantum.X, quantum.Zero))
np.allclose(output, quantum.Minus)

True

In [3]:
# Apply H-gates to the n input qubits
num_qubits = 2
qubits = quantum.Zero
for _ in range(num_qubits):
    qubits = np.kron(quantum.Zero, qubits)
    
Hs = quantum.H
for _ in range(num_qubits):
    Hs = np.kron(quantum.H, Hs)

qubits = np.dot(Hs, qubits)

In [4]:
# Run the black box
def black_box(a: np.array, mode: str) -> np.array:
    if mode == "constant":
        c = np.ones(a.shape, dtype=np.complex_)
        return quantum.normalize(c)
    if mode == "balanced":
        b = np.random.randint(2, size=a.shape)
        return quantum.normalize(b)
    raise ValueError("mode should be constant or balanced")
    
qubits = black_box(qubits, "constant")

In [5]:
# Apply H-gates to the n input qubits
qubits = np.dot(Hs, qubits)

In [6]:
quantum.info(qubits, "output")

output
shape: (8, 1)
[[ 1.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 [ 0.+0.j]]


When the oracle is constant, it has no effect (up to a global phase) on the input qubits, and the quantum states before and after querying the oracle are the same. Since the H-gate is its own inverse, in Step 4 we reverse Step 2 to obtain the initial quantum state of  |00...0⟩  in the first register.

After step 2, our input register is an equal superposition of all the states in the computational basis. When the oracle is balanced, phase kickback adds a negative phase to exactly half these states.